In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
# Data generators
train_datagen = ImageDataGenerator(
    zoom_range=0.5, shear_range=0.3, horizontal_flip=True, preprocessing_function=preprocess_input
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train = train_datagen.flow_from_directory('train_path', target_size=(256, 256), batch_size=32)
val = val_datagen.flow_from_directory('val_path', target_size=(256, 256), batch_size=32)

In [ ]:
# Load pre-trained VGG19 model without top layers
base_model = VGG19(input_shape=(256, 256, 3), include_top=False)
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
base_model.summary()

In [ ]:
# Custom head
X = Flatten()(base_model.output)
X = Dense(units=38, activation='softmax')(X)
model = Model(base_model.input, X)

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit_generator(train, steps_per_epoch=16, epochs=50, verbose=1, validation_data=val, validation_steps=16)

In [ ]:
# Evaluate the model
accuracy = model.evaluate_generator(val)[1]
print(f"The Accuracy of your model is {accuracy*100}%")


In [ ]:
# Class labels
class_labels = dict(zip(list(train.class_indices.values()), list(train.class_indices.keys())))

In [ ]:
# Prediction function
def predict_class(path):
    img = load_img(path, target_size=(256, 256))
    img_array = img_to_array(img)
    img_preprocessed = preprocess_input(img_array)
    img_expanded = np.expand_dims(img_preprocessed, axis=0)
    prediction = np.argmax(model.predict(img_expanded))
    print(f'The Image belongs to {class_labels[prediction]}')


In [ ]:
# Test prediction
test_image_path = 'test_image_path'
predict_class(test_image_path)